<a href="https://colab.research.google.com/github/ecostadelle/ProjetoPesquisaTratamentoIncertezas/blob/main/src/Transforma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns

# Arquivo de boletins
github = 'https://raw.githubusercontent.com/ecostadelle/ProjetoPesquisaTratamentoIncertezas/main/db/db.csv'

# Leitura do arquivo de boletins
boletins = pd.read_csv(github, decimal=',', sep=';')	



In [2]:
transforma = pd.DataFrame(
    columns=[
        'ano', 
        'periodo',
        'estudante', 
        'ira_nb1', 
        'ira_mr1', 
        'ira_nb3', 
        'rec_s1', 
        'rec_s2', 
        'qtd_disciplinas', 
        'ap', 
        'ac', 
        'rt', 
        'rf',
        'resultado_final',
        'situacao_atual'
    ]
)

anos_boletim = boletins['ano'].unique()

for ano in anos_boletim:
    alunos_ano = boletins[boletins['ano']==ano]['estudante'].unique()
    
    for aluno in alunos_ano:
        
        boletim_individual = boletins[(boletins['estudante']==aluno) & (boletins['ano']==ano)]
        periodo = int(boletim_individual[boletim_individual['estudante']==aluno]['periodo'].mode())

        # Lista com ano, estudante, ira no 1ºbim, ira após recuperação, ira após 3º bim, 
        #           qtd de recuperações no 1º sem, qtd de recuperações no 2º sem,
        #           qtd de disciplinas, aprovações, aprovações em conselho, retenções por nota, 
        #           retenções por falta 
        # calcula a quantidade de disciplinas
        
        qtd_disciplinas = len(boletim_individual)
        # calcula o IRA no 1º bimestre
        ira1 = pd.to_numeric(boletim_individual['nb1'], errors='coerce').fillna(0).mean().round(2)
        # calcula a soma das notas após a recuperação
        mr1_soma = pd.to_numeric(boletim_individual['mr1'], errors='coerce').fillna(0).sum()
        # calcula o IRA após a recuperação
        ira2 = (mr1_soma / qtd_disciplinas).round(2)        
        # calcula o soma das notas após o 3º bimestre
        nb3_soma = pd.to_numeric(boletim_individual['nb3'], errors='coerce').fillna(0).sum()
        # calcula o IRA após o 3º bimestre
        ira3 = ( (2*mr1_soma/3+ nb3_soma/3) / qtd_disciplinas).round(2)
        # calcula a quantidade de recuperações do 1º semestre
        rec_s1 = (pd.to_numeric(boletim_individual['ms1'], errors='coerce').fillna(0) < 60).sum(),
        # calcula a quantidade de recuperações do 2º semestre
        rec_s2 =  (pd.to_numeric(boletim_individual['ms2'], errors='coerce').fillna(0) < 60).sum(),
        # calcula a quantidade de disciplinas em que o estudante foi aprovado
        ap = (boletim_individual['resultado']=='AP').sum(),
        # calcula a quantidade de disciplinas em que o estudante foi aprovado com o recurso do conselho de classe
        ac = (boletim_individual['resultado']=='AC').sum(),
        # calcula a quantidade de disciplinas em que o estudante ficou retido por nota
        rt= (boletim_individual['resultado']=='RT').sum(),
        # calcula a quantidade de disciplinas em que o estudante ficou retido por falta
        rf = (boletim_individual['resultado']=='RF').sum(),
        # captura o resultado do ano do db
        resultado_ano = boletim_individual[boletim_individual['estudante']==aluno]['resultado_ano']
        # captura a situacao atual no db
        situacao_atual = boletim_individual[boletim_individual['estudante']==aluno]['situacao_atual']

        # produz um array com todos os dados de uma linha
        linha = [
            ano,
            periodo,
            aluno,
            ira1,
            ira2,
            ira3,
            float(rec_s1[0]),
            float(rec_s2[0]),
            qtd_disciplinas,
            float(ap[0]), 
            float(ac[0]), 
            float(rt[0]), 
            float(rf[0]),
            resultado_ano.values[0],
            situacao_atual.values[0]
        ]
        # insere uma nova linha após a última linha da tabela
        transforma.loc[len(transforma)] = linha

# Cassifica qualitativamente o ano de cada estudante, de modo que:
# - Grupo A - concluíram o ano letivo com êxito, sem recuperações;
# - Grupo B - concluíram o ano letivo com êxito, com pelo menos uma recuperação, sem aprovação por conselho de classe ou dependência;
# - Grupo C - concluíram o ano letivo com êxito, com pelo menos uma aprovação por conselho de classe ou dependência;
# - Grupo D - retidos por nota ou por falta em até 3 disciplinas; e
# - Grupo E - retidos por falta em 4 ou mais disciplinas.

for i in transforma.index:
    if ((transforma.loc[i, 'resultado_final']=='Aprovado')
        & 
        (transforma.loc[i, 'rec_s2'] + transforma.loc[i, 'rec_s1'] == 0)):
            transforma.loc[i, 'grupo'] = 'A'
    if ((transforma.loc[i, 'resultado_final']=='Aprovado')
        &
        (transforma.loc[i, 'ac'] + transforma.loc[i, 'rt'] + transforma.loc[i, 'rf'] == 0)
        &
        (transforma.loc[i, 'rec_s1'] + transforma.loc[i, 'rec_s2'] > 0)):
            transforma.loc[i, 'grupo'] = 'B'
    if ((transforma.loc[i, 'resultado_final']=='Aprovado') 
        & 
        (transforma.loc[i, 'ac'] + transforma.loc[i, 'rt'] > 0)):
            transforma.loc[i, 'grupo'] = 'C'
    if ((transforma.loc[i, 'resultado_final']=='Reprovado') 
        & 
        (transforma.loc[i, 'rf'] <= 3)):
            transforma.loc[i, 'grupo'] = 'D'
    if ((transforma.loc[i, 'resultado_final']=='Reprovado') 
        & 
        (transforma.loc[i, 'rf'] > 3)):
            transforma.loc[i, 'grupo'] = 'E'

# salva o arquivo agregado
transforma.to_csv('transforma.csv', sep=';', decimal=',', index=False)
